In [1]:
# Import libraries
import pandas as pd
import numpy as np

In [2]:
# Import dataset
filename = "2025-04-09+DATEN+OR+PRAKTKUM.xlsx"
df_terminals = pd.read_excel(filename, sheet_name = "Standorte inkl. Kapa´s")
df_postalcodes = pd.read_excel(filename, sheet_name = "PLZ inkl. Sendungsmengen")
df_distance_postalcodes_terminal = pd.read_excel(filename, sheet_name = "DISTANZEN PLZ-TERMINAL")
df_distance_two_terminals = pd.read_excel(filename, sheet_name = "DISTANZEN TERMINAL-TERMINAL")
df_shipmentvolume_postalcodes = pd.read_excel(filename, sheet_name = "Sendungsmenge PLZ zu PLZ")
df_adjacent_postalcodes = pd.read_excel(filename, sheet_name = "ADJAZENZ (BENACHBARTE PLZ)")
df_postalcodes_without_neighbors = pd.read_excel(filename, sheet_name = "PLZ ohne Nachbarn")

In [3]:
# Summary of the DataFrame - Terminals
df_terminals.info()
#df_terminals.describe().round(decimals=2)

# MAX. KAPA SE (# ZUSTELLUNGEN pro Tag) ---- translated as -----> Terminal max delivery capacity per day
# There is a fixed maximum capacity per location for inbound shipments (i.e., deliveries).
# Outbound shipments (pickups) can always be handled and are not capacity-limited.	

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 10 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Terminal code                               39 non-null     object 
 1   Terminal name                               39 non-null     object 
 2   Country ISO code                            39 non-null     object 
 3   X-DOCK ADDRESS: street and building number  39 non-null     object 
 4   X-DOCK ADDRESS: postal code                 39 non-null     int64  
 5   X-DOCK ADDRESS: city                        39 non-null     object 
 6   LAT                                         39 non-null     float64
 7   LON                                         39 non-null     float64
 8   Partner                                     39 non-null     object 
 9   MAX. KAPA SE (# ZUSTELLUNGEN pro Tag)       39 non-null     int64  
dtypes: float64(2), i

In [4]:
# Summary of the DataFrame - Postal Codes (PLZ)
df_postalcodes.info()

# ZUORDNUNG IST ---- translated as -----> Currently assigned Terminal by DHL (which is to be evaluated by Model for optimality).
# Ø SENDUNGEN ABH pro Tag ---- translated as -----> Shipments pickup per day
# Ø SENDUNGEN ZUS pro Tag ---- translated as ----->	Shipments delivery per day

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8183 entries, 0 to 8182
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   PLZ (TEXT)               8183 non-null   int64  
 1   PLZ (ZAHL)               8183 non-null   int64  
 2   Name                     8183 non-null   object 
 3   LAT                      8183 non-null   float64
 4   LON                      8183 non-null   float64
 5   ZUORDNUNG IST            8183 non-null   object 
 6   PARTNER                  8183 non-null   object 
 7   Ø SENDUNGEN ABH pro Tag  8183 non-null   float64
 8   Ø SENDUNGEN ZUS pro Tag  8183 non-null   float64
dtypes: float64(4), int64(2), object(3)
memory usage: 575.5+ KB


In [5]:
# Summary of the DataFrame - Distance between Postal Codes (PLZ) & Terminals
df_distance_postalcodes_terminal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319137 entries, 0 to 319136
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   von PLZ (TEXT)     319137 non-null  int64  
 1   von PLZ (ZAHL)     319137 non-null  int64  
 2   nach Terminalcode  319137 non-null  object 
 3   nach Terminal      319137 non-null  object 
 4   Distanz (km)       319137 non-null  float64
 5   Fahrzeit (min)     319137 non-null  float64
dtypes: float64(2), int64(2), object(2)
memory usage: 14.6+ MB


In [6]:
# Summary of the DataFrame - Distance between two Terminals
df_distance_two_terminals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1521 entries, 0 to 1520
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   von             1521 non-null   object 
 1   nach            1521 non-null   object 
 2   Distanz (km)    1521 non-null   float64
 3   Fahrzeit (min)  1521 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 47.7+ KB


In [7]:
# Assigning Terminal Codes to 'DataFrame - Distance between two Terminals' to maintain consistency in Index Columns
df_distance_two_terminals = df_distance_two_terminals.merge(df_terminals[["Terminal name", "Terminal code"]], how="left", left_on="von", right_on="Terminal name")
df_distance_two_terminals.drop(columns="Terminal name", inplace=True)
df_distance_two_terminals.rename(columns={"Terminal code": "Terminal code (von)"}, inplace=True)
df_distance_two_terminals = df_distance_two_terminals.merge(df_terminals[["Terminal name", "Terminal code"]], how="left", left_on="nach", right_on="Terminal name")
df_distance_two_terminals.drop(columns="Terminal name", inplace=True)
df_distance_two_terminals.rename(columns={"Terminal code": "Terminal code (nach)"}, inplace=True)
df_distance_two_terminals

,von,nach,Distanz (km),Fahrzeit (min),Terminal code (von),Terminal code (nach)
0,Aachen,Aachen,0.00,0,AAH,AAH
1,Aachen,Altentreptow,775.29,734,AAH,AQW
2,Aachen,Berlin,603.04,573,AAH,BER
3,Aachen,Bielefeld,241.30,236,AAH,BFE
4,Aachen,Bremen,376.90,363,AAH,BRE
...,...,...,...,...,...,...
1516,Worms,Tempelhof,602.71,590,ZQV,THF
1517,Worms,Unterschleißheim,385.12,364,ZQV,MUC
1518,Worms,Villingen-Schwenningen,253.32,253,ZQV,VIS
1519,Worms,Weissenhorn,274.24,264,ZQV,QUL


In [8]:
# Summary of the DataFrame - Shipment Volume Matrix: From PLZ to PLZ		
df_shipmentvolume_postalcodes.info()

# Ø SENDUNGEN ABH pro TagØ Anzahl Sendungen pro Tag (FEB 2025) ---- translated as -----> Average number of shipments per day for February 2025
# all other PLZ-to-PLZ combinations have no shipment volume during the given time period.	
# International EXPORT shipments (collection in Germany, delivery in another country) have the respective export gateway (terminal where the international scheduled service starts) as their destination. 
# International IMPORT shipments (collection in another country, delivery in Germany) have the respective import gateway (terminal in Germany where the international scheduled service ends) as their origin.		

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272508 entries, 0 to 272507
Data columns (total 3 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   von                                    272508 non-null  object 
 1   nach                                   272508 non-null  object 
 2   Ø Anzahl Sendungen pro Tag (FEB 2025)  272508 non-null  float64
dtypes: float64(1), object(2)
memory usage: 6.2+ MB


In [9]:
# Summary of the DataFrame - adjacent postal codes (PLZ) — essentially neighboring regions that are geographically close to each other.	
df_adjacent_postalcodes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47835 entries, 0 to 47834
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   PLZ              47835 non-null  int64
 1   benachbarte PLZ  47835 non-null  int64
dtypes: int64(2)
memory usage: 747.6 KB


In [10]:
# Summary of the DataFrame - postal codes (PLZ) that have no neighboring PLZ — meaning these are isolated regions without immediate geographic neighbors. 
df_postalcodes_without_neighbors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 1 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   PLZ ohne Nachbarn  25 non-null     int64
dtypes: int64(1)
memory usage: 332.0 bytes


In [16]:
# Using SCIP Solver
pip install pyscipopt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 27.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [18]:
# Import library
from pyscipopt import Model, quicksum

In [19]:
# Get Data

# Postal Codes List
Vp = df_postalcodes['PLZ (ZAHL)'].tolist() # Set of Postal Code areas (PLZs).
Vp_idx = range(len(Vp))
Vp_map = {plz: i for i, plz in enumerate(Vp)}

# Terminals List
Vt = df_terminals['Terminal code'].tolist() # Set of existing Terminal locations.
Vt_idx = range(len(Vt))
Vt_map = {t: j for j, t in enumerate(Vt)}

# Create connection cost dictionary: C[i][j] 
# Cost associated with assigning PLZ i to Terminal j (collection/distribution leg cost) and is a function of distance.
C = {}
for _, row in df_distance_postalcodes_terminal.iterrows():
    i = Vp_map[row['von PLZ (ZAHL)']]
    j = Vt_map[row['nach Terminalcode']]
    C[i, j] = row['Distanz (km)']

# Create linehaul cost dictionary: F[m][h] = cost for transport between Terminal m and Terminal h.
F = {}
for _, row in df_distance_two_terminals.iterrows():
    m = Vt_map[row['Terminal code (von)']]
    h = Vt_map[row['Terminal code (nach)']]
    F[m, h] = row['Distanz (km)']

# Delivery volume destined for PLZ i.
D = df_postalcodes['Ø SENDUNGEN ZUS pro Tag'].tolist() 

# Capacity limit of Terminal j.
B = df_terminals['MAX. KAPA SE (# ZUSTELLUNGEN pro Tag)'].tolist() 

# Set of neighboring PLZs for PLZ i, defined based on adjacency parameter Zik. N(i) = {k ∈ Vp | k=i,Zik = 1}.

df_adjacent_postalcodes['PLZ_idx'] = df_adjacent_postalcodes['PLZ'].map(Vp_map)
df_adjacent_postalcodes['benachbarte_idx'] = df_adjacent_postalcodes['benachbarte PLZ'].map(Vp_map)

# Drop rows where mapping failed (e.g., postal codes not found in Vp)
df_adjacent_postalcodes = df_adjacent_postalcodes.dropna(subset=['PLZ_idx', 'benachbarte_idx'])

# Convert float indices to int (map returns floats if NaN is possible)
df_adjacent_postalcodes['PLZ_idx'] = df_adjacent_postalcodes['PLZ_idx'].astype(int)
df_adjacent_postalcodes['benachbarte_idx'] = df_adjacent_postalcodes['benachbarte_idx'].astype(int)

# Group and build the adjacency dictionary
N = df_adjacent_postalcodes.groupby('PLZ_idx')['benachbarte_idx'].apply(list).to_dict()

In [ ]:
# Create model
model = Model("FacilityLocation")

# Set solver parameters
model.setRealParam("limits/time", 1800)  # Set time limit to 30 minutes
model.setRealParam("limits/gap", 0.05)  # Set optimality gap to 5%
model.setIntParam("display/verblevel", 3)  # Set verbosity to level 3 for detailed output


# Decision variables
x = {(i, j): model.addVar(vtype="BINARY") for i in Vp_idx for j in Vt_idx}
#  xij ∈ {0,1}: 1 if PLZ i is assigned to Terminal j, 0 otherwise.

y = {(i, k, m, h): model.addVar(vtype="BINARY") for i in Vp_idx for k in Vp_idx for m in Vt_idx for h in Vt_idx}
# yikmh ∈ {0,1}: 1 if flow occurs between terminals m and h due to an origin PLZ i assigned to m and a destination PLZ k assigned to h, 0 otherwise.

z = {i: model.addVar(vtype="BINARY") for i in Vp_idx}
# zi:  if a PLZ i has neighbors then zi = 1 else 0

# Objective: Minimize total cost
model.setObjective(
    quicksum(C[i, j] * x[i, j] for i, j in C) +
    quicksum(F[m, h] * y[i, k, m, h] for i, k, m, h in y),
    sense='minimize'
)

# Constraints:
# Single allocation constraint: every PLZ i is assigned to exactly one terminal j.
for i in Vp_idx:
    model.addCons(quicksum(x[i, j] for j in Vt_idx) == 1)

# Capacity constraint: total assigned pickup and delivery volume for any terminal j respects its capacity bj.
for j in Vt_idx:
    model.addCons(quicksum(D[i] * x[i, j] for i in Vp_idx) <= B[j])

# Connectivity constraint: if a PLZ i has neighbors (zi = 1) and is assigned to terminal j (xij = 1), 
                            # then at least one of its neighbors k ∈ N(i) must also be assigned to terminal j ( k∈N(i)xkj ≥ 1). 
                            # If zi = 0, the constraint becomes 0 ≥ 0 and is non-restrictive.
for i in Vp_idx:
    for j in Vt_idx:
       model.addCons(z[i] * quicksum(x[k, j] - x[i, j] for k in N[i]) >= 0)

# Linearize the relationship yikmh = xim ∧ xkh. 
# They ensure the linehaul cost fmh is considered in the objective if and only if 
# the origin PLZ i is assigned to terminal m and the destination PLZ k is assigned to terminal h.

for i in Vp_idx:
    for k in Vp_idx:
        for m in Vt_idx:
            for h in Vt_idx:
                model.addCons(y[i, k, m, h] <= x[i, m])
                model.addCons(y[i, k, m, h] <= x[k, h])
                model.addCons(y[i, k, m, h] >= x[i, m] + x[k, h] - 1)                

# Solve
model.optimize()

# Output
print("Objective value:", model.getObjVal())
print("Status:", model.getStatus())
#print("\n--- Solution ---")
#for j in Vt_idx:
#    for i in Vp_idx:
#        if model.getVal(x[i, j]) > 0.5:
#            print(f" - PLZ {Vp[i]} assigned to Terminal {Vt[j]}")